In [2]:
### Convert PNG to IntelHEX ###
from PIL import Image
import numpy as np
from intelhex import IntelHex
from io import StringIO

img = Image.open("image.png")
pix = np.array(img)
pix = np.ravel(pix, order='C') #flatten pixel info to 1-d array
ih = IntelHex() #new HEX objext
ih.frombytes(pix) #RGBA data to HEX object
ih.write_hex_file("decRGBA.hex",byte_count=32) #Memory location fix to 0x4000 needed using HEX editor

In [33]:
### Convert HEX to PNG, func1 ###
from intelhex import IntelHex
from PIL import Image
import numpy as np

# data=[None]*256
# for i in range(256):
#     data[i] = i
    #data[i] = f'ob{i:08b}'

ih = IntelHex()
ih.loadhex('memoutput_func1.hex')
pydict=ih.todict() #hex to dict
rgbcomp = np.fromiter(pydict.values(), dtype=int) #array of values in dict
uniqRgbComp = np.unique(rgbcomp) #int list of uniq colors
plte = np.zeros((len(uniqRgbComp),3)) #plte(24bit rgb) = uniqRgbComp(8bit rgb)
for i in range(len(uniqRgbComp)):
   plte[i] = np.array([ int(uniqRgbComp[i]&0b11100000), int( ((uniqRgbComp[i])&0b00011100) << 3 ), int( (uniqRgbComp[i]&0b00000011) << 6 ) ])
pltePix = np.zeros(len(rgbcomp)) #Pixel data points palette
for i in range(len(rgbcomp)): 
    pltePix[i] = np.argwhere(uniqRgbComp==rgbcomp[i])[0] #Match pixel data to palette
pltePix = np.reshape(pltePix, (640, 960)) #shows mached data in rectangular shape

im_RGB = Image.new(mode="P", size=(960,640)) #make new image
plte = np.ravel(plte) #flatten the array(to be compatible with .putpalette)
plte = np.uint8(plte).tolist() #change size of element(to be compatible with .putpalette)
im_RGB.putpalette(plte, rawmode="RGB") #make palette
pltePix = np.ravel(pltePix) #flatten the array of pixel data(to be compatible with .putdata)
im_RGB.putdata(pltePix) #make pixeldata of image
im_RGB.save("compRGB.png") #write image to file

# https://pillow.readthedocs.io/en/stable/handbook/concepts.html#modes - about pillow modes

In [7]:
### Test - Get 8-bit PNG's palette ###
from PIL import Image
import numpy as np

img8 = Image.open("..\\bmp\image-8.png")
pix8 = np.array(img8)
# pix8 = np.ravel(pix8, order='C')
# pix8
plte8 = img8.getpalette() #returns list
plte8 = np.array(plte8) #list to np
plte8 = np.reshape(plte8,(256,3)) #palette 구함, 일치확인

# 팔레트 만들어서->hex랑 rgb 매핑 -> 팔레트를 바탕으로 출력
# https://github.com/python-pillow/Pillow/issues/6263 - RGB to P
# https://stackoverflow.com/questions/67967366/how-to-write-image-with-palette-information - palette to image